In [ ]:
! pip install ase nglview


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# @title Visualización de moléculas { display-mode: "form" }


In [ ]:
#The database is here: https://wiki.fysik.dtu.dk/ase/ase/build/build.html
from ase.build import molecule
from ase.visualize import view

water = molecule('H2O')
FA = molecule('HCOOH')
C60 = molecule('C60')
cyclobutene = molecule('cyclobutene')
view(water, viewer='x3d')

In [ ]:
view(FA, viewer='x3d')

In [ ]:
view(C60, viewer='x3d')

In [ ]:
view(cyclobutene, viewer='x3d')

In [ ]:
# @title Contrucción de nanoribbons, nanotubos y superficies { display-mode: "form" }

In [ ]:
from ase.build import graphene_nanoribbon
from ase.visualize import view

gnr1 = graphene_nanoribbon(3, 4, type='armchair', saturated=True,
                               vacuum=3.5)
gnr2 = graphene_nanoribbon(2, 6, type='zigzag', saturated=True,
                           C_H=1.1, C_C=1.4, vacuum=3.0,
                           magnetic=True, initial_mag=1.12)
view(gnr1, viewer='ngl')

In [ ]:
view(gnr2, viewer='ngl')

In [ ]:
from ase.build import nanotube
from ase.visualize import view
from ase.io import write

cnt1 = nanotube(6, 0, length=4)
cnt2 = nanotube(3, 3, length=6, bond=1.4, symbol='Si')

view(cnt1, viewer='x3d')

In [ ]:
view(cnt2, viewer='x3d')

In [ ]:
from ase.build import fcc111
from ase.visualize import view
from ase.io import write

#https://wiki.fysik.dtu.dk/ase/ase/build/surface.html?highlight=fcc111#ase.build.fcc111

slab = fcc111('Al', size=(3,3,2))
view(slab,viewer='x3d')

In [ ]:
# ontop,fcc,bridge,hcp

# https://wiki.fysik.dtu.dk/ase/ase/build/surface.html?highlight=fcc111#ase.build.fcc111

from ase.build import fcc111,add_adsorbate
from ase.visualize import view
from ase.io import write

slab = fcc111('Al', size=(3,3,2))
add_adsorbate(slab,'H',1.5,'hcp')

view(slab,viewer='x3d')

In [ ]:
# @title Optimización { display-mode: "form" }

In [ ]:
#with the Powershell prompt
#cd Desktop
# ase gui H2O.traj
# show forces, bonds
from ase import Atoms
from ase.units import kJ,mol
from ase.build import molecule
from ase.optimize import BFGS, FIRE
from ase.calculators.emt import EMT
from ase.io import read
from ase.visualize import view
import numpy as np
import warnings
warnings.filterwarnings("ignore")

n2 = molecule('N2')

mu = n2.get_masses()[0] / 2 / mol * 0.001

natoms = len(n2)

n2.set_calculator(EMT())
dyn = BFGS(n2, trajectory='N2.traj')
dyn.run(fmax=0.001)

delta=0.001

n2.positions[0][2] = n2.get_positions()[0][2] + delta
g1 = abs(n2.get_forces()[0][2])

frec = 1 / (2 * np.pi) * np.sqrt(g1/delta / kJ * 1000 / mu) / 2.9979

print('Vibrational frequency (cm-1):',frec)

configs = read('N2.traj@0:')
view(configs, viewer='ngl')

In [ ]:
from ase import *
from ase.units import kB
from ase.optimize import LBFGS
from ase.calculators.emt import EMT
from ase.optimize.basin import BasinHopping
import numpy as np

d = 0.9575
t = np.pi / 180 * 104.51
water = Atoms('H2O',
              positions=[(d, 0, 0),
                         (d * np.cos(t), d * np.sin(t), 0),
                         (0, 0, 0)],
              calculator=EMT())

bh = BasinHopping(atoms=water,         # the system to optimize
                  temperature=100 * kB, # 'temperature' to overcome barriers
                  dr=0.5,               # maximal stepwidth
                  optimizer=LBFGS,      # optimizer to find local minima
                  fmax=0.1,             # maximal force for the optimizer
                  )

In [ ]:
#with the Powershell prompt
#cd Desktop
# ase gui H2O.traj
# show forces, bonds
from ase import Atoms
from ase.optimize import BFGS, FIRE
from ase.calculators.emt import EMT
from ase.io import read
from ase.visualize import view
import numpy as np
import warnings
warnings.filterwarnings("ignore")
d = 0.9575
t = np.pi / 180 * 104.51
water = Atoms('H2O',
              positions=[(d, 0, 0),
                         (d * np.cos(t), d * np.sin(t), 0),
                         (0, 0, 0)],
              calculator=EMT())
dyn = BFGS(water, trajectory='H2O.traj')
#dyn = FIRE(water, trajectory='H2O.traj')

dyn.run(fmax=0.05)
configs = read('H2O.traj@0:')
view(configs, viewer='ngl')

In [ ]:
# Test using mopac

from ase import Atoms
from ase.build import molecule
from ase.optimize import BFGS, FIRE
from ase.calculators.mopac import MOPAC
from ase.io import read
from ase.visualize import view
import numpy as np
import warnings
warnings.filterwarnings("ignore")

#Method 1: read xyz file
FA = read('FA.xyz')
FA.calc = EMT()

#Method 2: construct molecule with Atoms
d = 0.9575
t = np.pi / 180 * 104.51
water = Atoms('H2O',
              positions=[(d, 0, 0),
                         (d * np.cos(t), d * np.sin(t), 0),
                         (0, 0, 0)],
              calculator=EMT())


#Method 3: get the molecule from database
AcH = molecule('CH3COOH',calculator=EMT())

#Change the name of the sysmte below
dyn = BFGS(FA, trajectory='H2O.traj')

dyn.run(fmax=0.05)
configs = read('H2O.traj@0:')
view(configs, viewer='ngl')